In [11]:
# Import các thư viện
import os
import json
import faiss
import torch
import re
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer


In [12]:
index_dir = "data/faiss_index"

# Load mapping và documents
with open(os.path.join(index_dir, "documents.json"), "r") as f:
    documents = json.load(f)
with open(os.path.join(index_dir, "qa_mapping.json"), "r") as f:
    qa_mapping = json.load(f)

# Load FAISS index
index = faiss.read_index(os.path.join(index_dir, "qa_index.faiss"))
print(f"Loaded FAISS index with {index.ntotal} vectors")

# Load encoder và LLM
encoder_name = "all-MiniLM-L6-v2"
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
encoder = SentenceTransformer(encoder_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

def adjust_sentence(text):
    # Tách câu
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())

    if sentences and not re.search(r'[.!?]$', sentences[-1]):
        sentences = sentences[:-1]
    # Ghép lại
    result = ' '.join(sentences)
    # Thêm câu khuyến nghị
    if result:
        result = result.rstrip('.') + '.'
        result += "\nPlease consult with a qualified healthcare professional for accurate diagnosis and personalized medical advice."
    return result



Loaded FAISS index with 16402 vectors


In [23]:
query = input("Enter your medical question: ")
query

'How many people are affected by familial pityriasis rubra pilaris ?'

In [24]:
# Mã hóa câu hỏi thành vector
query_embedding = encoder.encode([query], convert_to_tensor=True)
query_embedding_np = query_embedding.cpu().numpy()
query_embedding_np.shape

(1, 384)

In [25]:
# Tìm kiếm trong Database k vector câu hỏi có Euclid-Distance gần nhất với vector câu hỏi từ người dùng.
top_k = 3
distances, indices = index.search(query_embedding_np, top_k)
print("Index các câu hỏi gần nhất:", indices[0])
print("Khoảng cách tương ứng:", distances[0])

Index các câu hỏi gần nhất: [7894 7896 7897]
Khoảng cách tương ứng: [0.42799622 0.5069239  0.5553884 ]


In [26]:
#Ánh xạ k câu hỏi tới k tài liệu tương ứng
retrieved_docs = [documents[idx] for idx in indices[0]]
retrieved_answers = [qa_mapping[idx]["completion"] for idx in indices[0]]
retrieved_info = list(zip(retrieved_docs, distances[0], retrieved_answers))

print("Retrieved Information:")
for i, (doc, dist, ans) in enumerate(retrieved_info):
    print(f"Reference {i + 1}:\n{doc}\n\nAnswer: {ans[:90]}...\n")

Retrieved Information:
Reference 1:
Topic: familial pityriasis rubra pilaris

Question: How many people are affected by familial pityriasis rubra pilaris ?

Answer: Familial pityriasis rubra pilaris is a rare condition. Its incidence is unknown, although ...

Reference 2:
Topic: familial pityriasis rubra pilaris

Question: Is familial pityriasis rubra pilaris inherited ?

Answer: Familial pityriasis rubra pilaris usually has an autosomal dominant inheritance pattern, w...

Reference 3:
Topic: familial pityriasis rubra pilaris

Question: What are the treatments for familial pityriasis rubra pilaris ?

Answer: These resources address the diagnosis or management of familial pityriasis rubra pilaris: ...



In [27]:
#Tạo prompt
context_parts = []
for i, (doc, dist, ans) in enumerate(retrieved_info):
    topic_part = doc.split("\n\n")[0].replace("Topic: ", "")
    context_parts.append(f"Reference {i + 1} on {topic_part}:\n{ans}")

context_str = "\n\n".join(context_parts)
messages = [
    {
        "role": "system",
        "content": f"""You are a medical assistant. Answer the user's question using ONLY the exact information from the provided references. Do not paraphrase, summarize, or add any explanations or examples that are not in the references. If you cannot find the exact words or phrases in the references, do not answer.

References:
{context_str}

Instructions:
1. Use only the exact words and phrases from the references.
2. Do not add any explanations or examples.
3. Do not mention the references explicitly.
4. If you finish answering, stop immediately.
"""
    },
    {
        "role": "user",
        "content": f"{query}"
    }
]

messages

[{'role': 'system',
  'content': "You are a medical assistant. Answer the user's question using ONLY the exact information from the provided references. Do not paraphrase, summarize, or add any explanations or examples that are not in the references. If you cannot find the exact words or phrases in the references, do not answer.\n\nReferences:\nReference 1 on familial pityriasis rubra pilaris:\nFamilial pityriasis rubra pilaris is a rare condition. Its incidence is unknown, although the familial form appears to be the least common type of pityriasis rubra pilaris.\n\nReference 2 on familial pityriasis rubra pilaris:\nFamilial pityriasis rubra pilaris usually has an autosomal dominant inheritance pattern, which means one copy of the altered gene in each cell is sufficient to cause the disorder. Affected individuals usually inherit the condition from one affected parent. However, the condition is said to have incomplete penetrance because not everyone who inherits the altered gene from a

In [28]:
#Tokenize prompt
input_ids = tokenizer.apply_chat_template(
    messages,
    return_tensors="pt",
    add_generation_prompt=True
).to(model.device)
input_ids.shape

torch.Size([1, 405])

In [29]:
with torch.no_grad():
    outputs = model.generate(
        input_ids = input_ids,
        max_new_tokens=512,
        temperature=0.1,
        do_sample=True,
        top_p=0.6,
        num_return_sequences=1,
        eos_token_id=model.config.eos_token_id,
    )
# Đưa input vào mô hình và lấy output
response = tokenizer.decode(outputs[0][input_ids.shape[1]:], skip_special_tokens=True)
response = adjust_sentence(response)
print("Answer:\n", response)


Answer:
 The number of people affected by familial pityriasis rubra pilaris is unknown, but it is described as being the least common type of pityriasis rubra pilaris.
Please consult with a qualified healthcare professional for accurate diagnosis and personalized medical advice.


In [40]:
#Compare with chatbot without context
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model2 = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Nhập câu hỏi
query = input("Enter your medical question: ")

prompt = f"""You are a highly knowledgeable medical assistant providing accurate and professional information to healthcare professionals.
Question: {query}

Instructions:
1. Provide a clear, concise, and well-organized medical answer.
2. Use appropriate medical terminology and maintain a professional tone.
3. Do not include any citation numbers, references, or footnote markers (e.g., [1], [2], etc.) under any circumstances.
4. Do not mention the references explicitly in your answer.


Answer:"""

# Tokenize và đưa vào model
inputs = tokenizer(prompt, return_tensors="pt").to(model2.device)

with torch.no_grad():
    outputs = model2.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.1,
        do_sample=True,
        top_p=0.6,
        num_return_sequences=1,
        eos_token_id=model2.config.eos_token_id,
    )

# Detokenize và in kết quả
response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
response = adjust_sentence(response)
print("\nAnswer (Without RAG):\n", response)



Answer (Without RAG):
 Adenylosuccinase deficiency is an inherited metabolic disorder that affects the body's ability to break down certain amino acids into their constituent parts. The primary symptoms of this condition typically manifest as muscle weakness, fatigue, and other neurological symptoms such as tremors, difficulty with coordination, and balance issues. Treatment options for adenylosuccinase deficiency depend on the specific genetic mutation causing the disorder and may involve:

1. Gene therapy: In some cases, gene therapy can be used to correct the underlying genetic defect responsible for the disease. This involves introducing healthy copies of the defective gene back into the patient's cells through a process called viral vector delivery. 2. Hormone replacement therapy: For patients who have been diagnosed with adenylosuccinase deficiency due to a hormonal imbalance, hormone replacement therapy may be necessary to restore normal levels of hormones like cortisol and ald